In [1]:
import pandas as pd

In [2]:
data = pd.read_excel('IR_Spring2021_ph12_7k.xlsx')

In [1]:
#normalize 1 method
def normalize1(myList):
    """"
    we should remove some specific characters eg: we do not save dot in the end of sentence with the last word
    we just need to remove if these specific chars are at the last of our word
    """
    for i in range(len(myList)):
        if myList[i].endswith('.'):
            myList[i] = myList[i].replace('.','')
        elif myList[i].endswith('،'):
            myList[i] = myList[i].replace('،','')
        elif myList[i].endswith(':'):
            myList[i] = myList[i].replace(':','')
        else:
            pass
    #the two last members means end of message and it is in all docs so we do not need it
    myList.pop()
    myList.pop()

In [2]:
#class below is for holding single id and single url
class holder:
    def __init__(self,ID = 0,url = None):
        self.ID = ID
        self.url = url
    def __eq__(self,obj):
        return str(self.ID)+self.url == str(obj.ID)+obj.url
    def __str__(self):
        return str(self.ID)+' '+self.url

In [3]:
#class below is for holding term frequency and list of holder classess
class termValue:
    def __init__(self):
        self.frequency = 0
        self.holderList = []
    def addHolder(self,newHolder):
        if newHolder not in self.holderList:
            self.holderList.append(newHolder)

In [4]:
#function below if for creating our dictionary
def createDictionary(myData):
    #dictionary data structure below is for holding term and frequency and holderLists
    #finally our form is like {term:(freq,[listOfHolders])}
    #dictionary[term] is termValue class object that contains freq+lisOfHolders that is object of holder class
    termDictionary = {}
    for counter in range(len(myData)):
        x = myData['content'][counter].split()
        y = myData['id'][counter]
        z = myData['url'][counter]
        normalize1(x)
        for i in x:
            if i not in termDictionary:
                termDictionary[i] = termValue()
                myHolder = holder(y,z)
                termDictionary[i].addHolder(myHolder)
            else:
                myHolder = holder(y,z)
                termDictionary[i].addHolder(myHolder)
    return termDictionary

In [11]:
termDictionary = createDictionary(data)

In [5]:
#function below is for calculating frequency of each term
def frequencyCalc(myDictionary):
    for i in myDictionary:
        myDictionary[i].frequency = len(myDictionary[i].holderList)
frequencyCalc(termDictionary)

In [6]:
#function below is for printing our whole dictionary
def printDictionary(myDictionary):
    for i in myDictionary:
        print('term:',i)
        print('frequency:',myDictionary[i].frequency)
        print('--postings--')
        for j in myDictionary[i].holderList:
            print(j)
        print('-----------------------------------------------------------')
printDictionary(termDictionary)

In [29]:
#algorithm below is for writing our dictionary to text file
textFile = open('dictionary.txt','a',encoding = 'UTF-8')
for i in termDictionary:
    for j in termDictionary[i].holderList:
        textFile.write(str(i)+'$'+str(j.ID)+'$'+str(j.url)+'\n')
textFile.close()

In [6]:
#algorithm below is for retrieval of dictionary
myFile = open('dictionary.txt',encoding = 'UTF-8')
theDictionary = {}
for i in myFile:
    res = i.split('$')
    if res[0] not in theDictionary:
        theDictionary[res[0]] = termValue()
        myHolder = holder(res[1],res[2])
        theDictionary[res[0]].addHolder(myHolder)
    else:
        myHolder = holder(res[1],res[2])
        theDictionary[res[0]].addHolder(myHolder)
myFile.close()

In [7]:
frequencyCalc(theDictionary)

In [9]:
def merge(term1,term2,myDictionary):
    answerList = []
    if term1 not in myDictionary or term2 not in myDictionary:
        print('no result')
    else:
        term1List = myDictionary[term1].holderList
        term2List = myDictionary[term2].holderList
        len1 = len(term1List)
        len2 = len(term2List)
        i,j = 0,0
        while i < len1 and j < len2:
            if term1List[i].ID == term2List[j].ID:
                answerList.append(term1List[i])
                i += 1
                j += 1
            elif term1List[i].ID < term2List[j].ID:
                i += 1
            else:
                j += 1
    return answerList

In [10]:
def tripleMerge(term1,term2,term3,myDictionary):
    answerList = []
    if term1 not in myDictionary or term2 not in myDictionary or term3 not in myDictionary:
        print('no result')
    else:
        term1List = myDictionary[term1].holderList
        term2List = myDictionary[term2].holderList
        term3List = myDictionary[term3].holderList
        len1 = len(term1List)
        len2 = len(term2List)
        len3 = len(term3List)
        i,j,k = 0,0,0
        while i < len1 and j < len2 and k < len3:
            if term1List[i].ID == term2List[j].ID == term3List[k].ID:
                answerList.append(term1List[i])
                i += 1
                j += 1
                k += 1
            elif term1List[i].ID < term2List[j].ID and term1List[i].ID < term3List[k].ID:
                i += 1
            elif term2List[j].ID < term1List[i].ID and term2List[j].ID < term3List[k].ID:
                j += 1
            elif term3List[k].ID < term1List[i].ID and term3List[k].ID < term2List[j].ID:
                k += 1
            elif term1List[i].ID == term2List[j].ID and term1List[i].ID < term3List[k].ID:
                i += 1
                j += 1
            elif term1List[i].ID == term3List[k].ID and term1List[i].ID < term2List[j].ID:
                i += 1
                k += 1
            else:
                j += 1
                k += 1
    return answerList

In [11]:
def engine(myDictionary):
    res = []
    query = input('enter query: ')
    queryList = query.split()
    queryListLen = len(queryList)
    if queryListLen == 1:
        res = merge(queryList[0],queryList[0],myDictionary)
        for i in res:
            print(i)
    elif queryListLen == 2:
        print('--having both--')
        res = merge(queryList[0],queryList[1],myDictionary)
        for i in res:
            print(i)
        print('--having just one term--')
        res = merge(queryList[0],queryList[0],myDictionary)
        for i in res:
            print(i)
        res = merge(queryList[1],queryList[1],myDictionary)
        for i in res:
            print(i)
    elif queryListLen == 3:
        print('--having three terms--')
        res = tripleMerge(queryList[0],queryList[1],queryList[1],myDictionary)
        for i in res:
            print(i)
        print('--having two terms--')
        res = merge(queryList[0],queryList[1],myDictionary)
        for i in res:
            print(i)
        res = merge(queryList[0],queryList[2],myDictionary)
        for i in res:
            print(i)
        res = merge(queryList[1],queryList[2],myDictionary)
        for i in res:
            print(i)
        print('--having just one term--')
        res = merge(queryList[0],queryList[0],myDictionary)
        for i in res:
            print(i)
        res = merge(queryList[1],queryList[1],myDictionary)
        for i in res:
            print(i)
        res = merge(queryList[2],queryList[2],myDictionary)
        for i in res:
            print(i)
    else:
        print('this engine does not support....')

In [13]:
engine(theDictionary)

enter query: استقلال تهران
--having both--
75 https://www.isna.ir/news/99020805715/با-دستور-سلطانی-فر-کمپ-حجازی-به-استقلال-واگذار-می-شود

--having just one term--
10 https://www.isna.ir/news/99010401821/پزشک-و-رئیس-هیأت-مدیره-استقلال-مشکوک-به-کرونا

26 https://www.isna.ir/news/99010904082/آخرین-وضعیت-مدیرعاملی-استقلال-و-پرسپولیس-تعیین-نفر-اول-آبی-ها

75 https://www.isna.ir/news/99020805715/با-دستور-سلطانی-فر-کمپ-حجازی-به-استقلال-واگذار-می-شود

81 https://www.isna.ir/news/99021309405/معاون-سازمان-خصوصی-سازی-از-حضور-هواداران-در-باشگاه-استقلال

111 https://www.isna.ir/news/99023021921/احتمال-برگزاری-اردوی-فولاد-در-تهران-رونمایی-از-ماسک-فولاد

113 https://www.isna.ir/news/99030100481/فولادی-ها-تست-کرونا-دادند

119 https://www.isna.ir/news/99030100481/فولادی-ها-تست-کرونا-دادند

120 https://www.isna.ir/news/99030301699/تست-کرونای-فولادی-ها-منفی-بود-آغاز-تمرینات-شاگردان-نکونام-پس

124 https://www.isna.ir/news/99030301536/احتمال-آغاز-تمرینات-نفت-مسجدسلیمان-از-فردا-در-تهران

134 https://www.isn